In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.contingency_tables as ct

In [ ]:
benchmark_data_path = "../resources/BenchmarkDatasetClean/"
benchmark_dataset = os.listdir(benchmark_data_path)
benchmark_save_path = "../resources/BenchmarkResults_29_9_2023_Macro_Tests_all_noise_redo_redo"
benchmark_save_path2 = "../resources/BenchmarkResults_23_9_2023_Macro_Tests_HTML_original"
benchmark_save_path1 = "../resources/BenchmarkResults_23_9_2023_Macro_Tests_new_format_added_test_redo/"

benchmark_dataset = [data for data in benchmark_dataset if data.endswith(
    ".csv") and "breast" not in data]
dfs = []
for bd in benchmark_dataset:

    pass_metrics = []
    print(bd)
    df1 = pd.read_csv(os.path.join(benchmark_save_path1, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df2 = pd.read_csv(os.path.join(benchmark_save_path2, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df3 = pd.read_csv(os.path.join(benchmark_save_path, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df2["precision_per_cell_correctness_top1"] = df2["precision_per_cell_correctness_top1"]*0.01
    df2["recall_per_cell_correctness_top1"] = df2["recall_per_cell_correctness_top1"]*0.01
    df3["precision_per_cell_correctness_top1"] = df3["precision_per_cell_correctness_top1"]*0.01
    df3["recall_per_cell_correctness_top1"] = df3["recall_per_cell_correctness_top1"]*0.01
    df2["TableManipulation"] = "OriginalData"
    df1["TableManipulation"] = "OriginalData"
    df = pd.concat([df1, df2, df3], ignore_index=True)
    print(df1.shape, df2.shape, df.shape)
    df["Dataset-name"] = bd
    df["F1Score_per_cell_correctness_top1"] = df.apply(lambda x: np.round(2*(x.recall_per_cell_correctness_top1*x.precision_per_cell_correctness_top1)/(
        x.recall_per_cell_correctness_top1+x.precision_per_cell_correctness_top1)) if (x.recall_per_cell_correctness_top1*x.precision_per_cell_correctness_top1) != 0 else 0, axis=1)
    dfs.append(df)
all_data = pd.concat(dfs, ignore_index=True)

attributes = {c: list(df[c].unique()) for c in df.columns if c in [
    "temperature", "tableFormat", "table_ops", "TestCase"]}

#### RQ1

In [ ]:
test_cases = ["TableColumnReorderTests",
              "TableReconstructionTests1", "TableTransposeTests"]
formats = all_data["tableFormat"].unique()

In [ ]:
all_data_test_vs_format_on_all_noise_ops = all_data[(
    all_data["temperature"] == 0.0)]  
print(all_data_test_vs_format_on_all_noise_ops.shape)
test_vs_format_on_all_noise_ops = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["tableFormat"],
    columns='TestCase',
    values=["F1Score_per_cell_correctness_top1", 'recall_per_cell_correctness_top1',
            "precision_per_cell_correctness_top1"],  
    aggfunc=["mean"]  # Custom aggregation function to append values to a list
)
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops*100
test_vs_format_on_all_noise_ops = np.round(
    test_vs_format_on_all_noise_ops, decimals=2)
cols = [("mean", f"{metric}_per_cell_correctness_top1", T)
        for T in test_cases for metric in ["precision", "recall", "F1Score"]]
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops[cols]
indi = ['CommaSeparatedFormat', 'DFloaderFormat', 'DataMatrixFormat',
        'JsonFormat', 'MarkdownFormat',
        'TabSeparatedFormat', 'HTMLFormat', 'HTMLNoSpaceFormat']
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops.reindex(indi)
test_vs_format_on_all_noise_ops

In [ ]:
cols_f1 = [("mean", f"{metric}_per_cell_correctness_top1", T)
           for T in test_cases for metric in ["F1Score"]]
test_vs_format_on_all_noise_ops_f1 = test_vs_format_on_all_noise_ops[cols_f1]
test_vs_format_on_all_noise_ops_f1

#### P-values RQ1 Macro

In [ ]:
all_data_test_vs_format_on_all_noise_ops = all_data[(
    all_data["temperature"] == 0.0)]  
print(all_data_test_vs_format_on_all_noise_ops.shape)
test_vs_format_on_all_noise_ops = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["TestCase"],
    columns='tableFormat',
    values=["F1Score_per_cell_correctness_top1"],
    # Custom aggregation function to append values to a list
    aggfunc=lambda x: list(x)
)

In [ ]:
import scipy

p_val = {}
for id in range(test_vs_format_on_all_noise_ops.shape[0]):
    print(id)
    if test_vs_format_on_all_noise_ops.index[id] != "TableReconstructionTests":
        print(
            f"For Test: {test_vs_format_on_all_noise_ops.index[id]} pass@1 temp =0.1")
        sorted_list = [i for i in test_vs_format_on_all_noise_ops_f1.sort_values(
            by=[("mean", "F1Score_per_cell_correctness_top1", test_vs_format_on_all_noise_ops.index[id])], ascending=False).index]
        print(sorted_list)
        sub_vals = {}
        index_val = test_vs_format_on_all_noise_ops.index[id]
        for x in range(1, len(sorted_list)):
            vals1 = test_vs_format_on_all_noise_ops.loc[index_val, (
                "F1Score_per_cell_correctness_top1", sorted_list[0])]
            vals2 = test_vs_format_on_all_noise_ops.loc[index_val, (
                "F1Score_per_cell_correctness_top1", sorted_list[x])]
            min_val = min(len(vals1), len(vals2))
            print(min_val)
            p_value = scipy.stats.ttest_rel(vals1[:min_val], vals2[:min_val])
            sub_vals[f"{sorted_list[0]}-{sorted_list[x]}"] = {"p-value": p_value.pvalue,
                                                              "statistics": p_value.statistic, "df": p_value.df, "tests_count": min_val}
        p_val[test_vs_format_on_all_noise_ops.index[id]] = sub_vals

In [ ]:
import json
import datetime


class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, pd.Series):
            return obj.tolist()
        elif isinstance(obj, datetime):
            # Handle datetime objects
            return obj.isoformat()
        elif isinstance(obj, set):
            # Handle sets
            return list(obj)
        return super().default(obj)


with open("../resources/all_p_vals/p_vals_macro_tests_RQ1.json", "w") as f:
    json.dump(p_val, f, indent=3, cls=CustomJSONEncoder)